In [1]:
import numpy as np
import open3d as o3d
import os

from data import (
    load_point_cloud,
    np_point_cloud2_pcd,
    pcd2np_point_cloud,
    write_las_file,
)
from fusion import weighted_fusion_filter
from weights import compute_weights


VOXEL_SIZE = 0.05
OUTPUT_DIR = "output"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
POINT_CLOUDS = ["assets/bun000_Cloud.las", "assets/bun045_Cloud.las"]

# load
point_clouds_pcd = [np_point_cloud2_pcd(load_point_cloud(pc)) for pc in POINT_CLOUDS]

# merge
reference_pcd = o3d.geometry.PointCloud()
for pcd in point_clouds_pcd:
    reference_pcd.points.extend(pcd.points)
    reference_pcd.colors.extend(pcd.colors)
    reference_pcd.normals.extend(pcd.normals)

# downsampling (voxelization)
reference_pcd_ds = reference_pcd.voxel_down_sample(voxel_size=VOXEL_SIZE)
reference_pcd_ds_sub = reference_pcd.voxel_down_sample(voxel_size=VOXEL_SIZE / 2)

print("Number of points in search space (voxels): ", len(reference_pcd_ds.points))

Number of points in search space (voxels):  25


In [3]:
POINT_CLOUDS_ACCURACY = [0.01, 0.01]

# global weights
print("Computing point cloud weights ... ", end="")

global_weights = compute_weights(
    reference_pcd,
    point_clouds_pcd,
    POINT_CLOUDS_ACCURACY,
    voxel_size=VOXEL_SIZE,
)

print("Done!")
print("Global Weights:", global_weights)

Computing point cloud weights ... Done!
Global Weights: [0.05143779 0.0116153 ]


In [4]:
THRESHOLD = 0.01
K_GLOBAL = 1
K_LOCAL = 1

reference_points = np.asarray(reference_pcd_ds.points)

print("Fuse point clouds ... ")

fused_pcd = weighted_fusion_filter(
    point_clouds_pcd,
    global_weights,
    reference_points,
    reference_pcd_ds_sub,
    VOXEL_SIZE,
    THRESHOLD,
    K_GLOBAL,
    K_LOCAL,
)

# FIXME: this is super inefficient and the fused pcd contains only about 10% unique points
fused_pc = np.unique(pcd2np_point_cloud(fused_pcd), axis=0)

Fuse point clouds ... 


100%|██████████| 25/25 [00:01<00:00, 21.10it/s]


In [5]:
file_name = "weighted_fused_filtered_{}_cm_vox.las".format(int(VOXEL_SIZE * 100))
write_las_file(fused_pc, os.path.join(OUTPUT_DIR, file_name))